In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                             AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier,
                             VotingClassifier, StackingClassifier)
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.experimental import enable_hist_gradient_boosting
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from concurrent.futures import ThreadPoolExecutor
import time
import warnings
warnings.filterwarnings('ignore')

def compare_ml_models(X, y, test_size=0.2, random_state=42, cv=5):
    """
    Compare multiple machine learning models simultaneously and display results
    
    Parameters:
    X: Input features
    y: Target labels
    test_size: Test data size ratio
    random_state: Random seed for reproducibility
    cv: Number of cross-validation folds
    
    Returns:
    DataFrame containing results of all models
    """
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Define models
    models = {
        # Linear Models
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=random_state),
        'Ridge Classifier': RidgeClassifier(random_state=random_state),
        'SGD Classifier': SGDClassifier(random_state=random_state),
        
        # SVM Models
        'Linear SVM': LinearSVC(random_state=random_state),
        'RBF SVM': SVC(kernel='rbf', probability=True, random_state=random_state),
        'Polynomial SVM': SVC(kernel='poly', probability=True, random_state=random_state),
        
        # Tree-based Models
        'Decision Tree': DecisionTreeClassifier(random_state=random_state),
        'Random Forest': RandomForestClassifier(random_state=random_state),
        'Extra Trees': ExtraTreesClassifier(random_state=random_state),
        'Gradient Boosting': GradientBoostingClassifier(random_state=random_state),
        'Hist Gradient Boosting': HistGradientBoostingClassifier(random_state=random_state),
        
        # Boosting Algorithms
        'AdaBoost': AdaBoostClassifier(random_state=random_state),
        'XGBoost': XGBClassifier(random_state=random_state, verbosity=0),
        'LightGBM': LGBMClassifier(random_state=random_state, verbose=-1),
        'CatBoost': CatBoostClassifier(random_state=random_state, verbose=0),
        
        # Bayesian Models
        'Gaussian NB': GaussianNB(),
        'Bernoulli NB': BernoulliNB(),
        
        # Nearest Neighbors
        'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
        'KNN (k=5)': KNeighborsClassifier(n_neighbors=5),
        'KNN (k=7)': KNeighborsClassifier(n_neighbors=7),
        
        # Neural Networks
        'MLP (1 layer)': MLPClassifier(hidden_layer_sizes=(100,), random_state=random_state, max_iter=1000),
        'MLP (2 layers)': MLPClassifier(hidden_layer_sizes=(100, 50), random_state=random_state, max_iter=1000),
        
        # Discriminant Analysis
        'LDA': LinearDiscriminantAnalysis(),
        'QDA': QuadraticDiscriminantAnalysis(),
        
        # Ensemble Methods
        'Bagging': BaggingClassifier(random_state=random_state),
        
        # Gaussian Process
        'Gaussian Process': GaussianProcessClassifier(random_state=random_state),
    }
    
    results = []
    
    def train_and_evaluate(model_name, model):
        try:
            start_time = time.time()
            
            # Train model
            model.fit(X_train, y_train)
            
            # Predictions
            y_pred = model.predict(X_test)
            
            # Calculate evaluation metrics
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
            recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
            
            # Calculate ROC AUC if possible
            try:
                if hasattr(model, "predict_proba"):
                    y_proba = model.predict_proba(X_test)
                    if y_proba.shape[1] > 2:  # Multi-class
                        roc_auc = roc_auc_score(y_test, y_proba, multi_class='ovr')
                    else:
                        roc_auc = roc_auc_score(y_test, y_proba[:, 1])
                else:
                    roc_auc = np.nan
            except:
                roc_auc = np.nan
            
            # Calculate cross-validation score
            cv_scores = cross_val_score(model, X, y, cv=cv, n_jobs=1)
            cv_mean = np.mean(cv_scores)
            cv_std = np.std(cv_scores)
            
            training_time = time.time() - start_time
            
            return {
                'Model': model_name,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1 Score': f1,
                'ROC AUC': roc_auc,
                'CV Mean Score': cv_mean,
                'CV Std': cv_std,
                'Training Time (s)': training_time
            }
            
        except Exception as e:
            print(f"Error training {model_name}: {str(e)}")
            return {
                'Model': model_name,
                'Accuracy': np.nan,
                'Precision': np.nan,
                'Recall': np.nan,
                'F1 Score': np.nan,
                'ROC AUC': np.nan,
                'CV Mean Score': np.nan,
                'CV Std': np.nan,
                'Training Time (s)': np.nan
            }
    
    # Execute models in parallel
    with ThreadPoolExecutor(max_workers=min(len(models), 8)) as executor:
        futures = [executor.submit(train_and_evaluate, name, model) for name, model in models.items()]
        results = [future.result() for future in futures]
    
    # Create results table
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by='F1 Score', ascending=False)
    
    return results_df


# Example usage:
from sklearn.datasets import load_iris
data = load_iris()
X, y = data.data, data.target
results = compare_ml_models(X, y)
print(results)